In [67]:
import os
import random
import fnmatch
import datetime
import pickle

# data processing
import numpy as np
np.set_printoptions(formatter={'float_kind':lambda x: "%.4f" % x})

import pandas as pd
pd.set_option('display.width', 300)
pd.set_option('display.float_format', '{:,.4f}'.format)
pd.set_option('display.max_colwidth', 200)

# tensorflow
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential  # V2 is tensorflow.keras.xxxx, V1 is keras.xxx
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
print( f'tf.__version__: {tf.__version__}' )
print( f'keras.__version__: {keras.__version__}' )

# sklearn
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split


%matplotlib inline
from PIL import Image

tf.__version__: 2.4.1
keras.__version__: 2.3.1


In [45]:
import os
import fnmatch
import csv

data_dir = "C:/Users/User/Desktop/MLiS project/training_data/training_data"
#data_dir = '/This PC/Desktop/MLiS project/training_data/training_data'
file_list = os.listdir(data_dir)
image_paths = []
pattern = "*.png"
for filename in file_list:
    if fnmatch.fnmatch(filename, pattern):
        image_paths.append(os.path.join(data_dir,filename))
        angle = int(filename[-7:-4])

image_id = []
angle = []
speed = []

data_open = False

try:
    input_file = open('training_norm.csv','r')
    data_open = True
    
except:
    print('Unable to open file.')
    
if data_open:

    array = np.zeros((0,3))

    for line in input_file:
        entries = line.split(',')
        temp = np.array([float(entries[0]), float(entries[1]), float(entries[2])])
        array = np.vstack((array, temp))

    input_file.close()

for i in range(13798):
    image_id.append(array[i][0])
    angle.append(array[i][1])
    speed.append(array[i][2])


In [47]:
image_paths

['C:/Users/User/Desktop/MLiS project/training_data/training_data\\1.png',
 'C:/Users/User/Desktop/MLiS project/training_data/training_data\\10.png',
 'C:/Users/User/Desktop/MLiS project/training_data/training_data\\100.png',
 'C:/Users/User/Desktop/MLiS project/training_data/training_data\\1000.png',
 'C:/Users/User/Desktop/MLiS project/training_data/training_data\\10000.png',
 'C:/Users/User/Desktop/MLiS project/training_data/training_data\\10001.png',
 'C:/Users/User/Desktop/MLiS project/training_data/training_data\\10002.png',
 'C:/Users/User/Desktop/MLiS project/training_data/training_data\\10003.png',
 'C:/Users/User/Desktop/MLiS project/training_data/training_data\\10004.png',
 'C:/Users/User/Desktop/MLiS project/training_data/training_data\\10005.png',
 'C:/Users/User/Desktop/MLiS project/training_data/training_data\\10006.png',
 'C:/Users/User/Desktop/MLiS project/training_data/training_data\\10007.png',
 'C:/Users/User/Desktop/MLiS project/training_data/training_data\\10008.pn

In [68]:
def nvidia_model():
    model = Sequential(name='Nvidia_Model')
    
    # elu=Expenential Linear Unit, similar to leaky Relu
    # skipping 1st hiddel layer (nomralization layer), as we have normalized the data
    
    # Convolution Layers
    model.add(Conv2D(24, (5, 5), strides=(2, 2), input_shape=(66, 200, 3), activation='elu')) 
    model.add(Conv2D(36, (5, 5), strides=(2, 2), activation='elu')) 
    model.add(Conv2D(48, (5, 5), strides=(2, 2), activation='elu')) 
    model.add(Conv2D(64, (3, 3), activation='elu')) 
    model.add(Dropout(0.2)) # not in original model. added for more robustness
    model.add(Conv2D(64, (3, 3), activation='elu')) 
    
    # Fully Connected Layers
    model.add(Flatten())
    model.add(Dropout(0.2)) # not in original model. added for more robustness
    model.add(Dense(100, activation='elu'))
    model.add(Dense(50, activation='elu'))
    model.add(Dense(10, activation='elu'))
    
    # output layer: turn angle (from 45-135, 90 is straight, <90 turn left, >90 turn right)
    model.add(Dense(1)) 
    
    # since this is a regression problem not classification problem,
    # we use MSE (Mean Squared Error) as loss function
    optimizer = Adam(lr=1e-3) # lr is learning rate
    model.compile(loss='mse', optimizer=optimizer)
    
    return model

model = nvidia_model()
print(model.summary())

Model: "Nvidia_Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 31, 98, 24)        1824      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 47, 36)        21636     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 5, 22, 48)         43248     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 20, 64)         27712     
_________________________________________________________________
dropout (Dropout)            (None, 3, 20, 64)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 1, 18, 64)         36928     
_________________________________________________________________
flatten (Flatten)            (None, 1152)             